In [ ]:
!pip install nlp2go transformers git+https://github.com/Maluuba/nlg-eval.git@master

In [ ]:
from nlgeval import NLGEval

nlgeval = NLGEval(
    metrics_to_omit=['METEOR', 'EmbeddingAverageCosineSimilairty', 'SkipThoughtCS', 'VectorExtremaCosineSimilarity',
                     'GreedyMatchingScore', 'CIDEr'])

In [ ]:
!wget https://github.com/voidful/BDG/releases/download/v2.0/BDG.pt
!wget https://github.com/voidful/BDG/releases/download/v2.0/BDG_ANPM.pt
!wget https://github.com/voidful/BDG/releases/download/v2.0/BDG_PM.pt

In [ ]:
from transformers import RobertaTokenizer
from transformers import RobertaForMultipleChoice
import torch
from torch.distributions import Categorical
import itertools as it
import nlp2go

tokenizer = RobertaTokenizer.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")
model = RobertaForMultipleChoice.from_pretrained("LIAMF-USP/roberta-large-finetuned-race")
model.eval()
model.to("cuda")

dg_model = nlp2go.Model('./BDG.pt')
dg_model_pm = nlp2go.Model('./BDG_PM.pt')
dg_model_both = nlp2go.Model('./BDG_ANPM.pt')

In [ ]:
context = """There are many things we need to know that we do not learn at school . For example , if we want to use our money wisely , we need to shop carefully . We need to know how to compare the prices of things in different shops . We need to be able to compare the quality of different brands . We need to know how to make a choice when we shop . Knowing how to make such choices is a " life skill " , and we need these skills if we are to live useful and happy lives . Some of these choices are small . For example , will I take an apple for lunch or a pear ? Will I go to school by bus or on foot ? Will I wear the red T - shirt or the blue one to the movies ? Other choices are more important . For example , will I eat healthy food for lunch or will eat junk food because it is tastier ? Will I work hard in all my classes or will I only work hard in the classes I enjoy ? We make choices like this every day . We have to realize that the choices we make can affect the rest of our lives . Just as importantly , our choices can also affect other people . The next time you decide to waste time in class , play a joke on someone or talk loudly at the movies , think about this : who else does your choice affect ?"""
question = """ "We need " life skills """
answer = "to live useful and happy lives"

In [ ]:
d_input = context + '</s>' + question + '</s>' + answer
choices = dg_model.predict(d_input, decodenum=3)['result']
choices_pm = dg_model_pm.predict(d_input, decodenum=3)['result']
choices_both = dg_model_both.predict(d_input, decodenum=3)['result']
all_options = choices + choices_pm + choices_both

In [ ]:
def selection(context, question, answer, all_options):
    max_combin = [0, []]
    for combin in set(it.combinations(all_options, 3)):
        options = list(combin) + [answer]
        keep = True
        for i in set(it.combinations(options, 2)):
            a = "".join([char if char.isalpha() or char == " " else " " + char + " " for char in i[0]])
            b = "".join([char if char.isalpha() or char == " " else " " + char + " " for char in i[1]])
            metrics_dict = nlgeval.compute_individual_metrics([a], b)
            if metrics_dict['Bleu_1'] > 0.5:
                keep = False
                break
        if keep:
            prompt = context + tokenizer.sep_token + question
            encoding_input = []
            for choice in options:
                encoding_input.append([prompt, choice])
            encoding_input.append([prompt, answer])
            labels = torch.tensor(len(options) - 1).unsqueeze(0)
            encoding = tokenizer(encoding_input, return_tensors='pt', padding=True, truncation='only_first')
            outputs = model(**{k: v.unsqueeze(0).to('cuda') for k, v in encoding.items()},
                            labels=labels.to('cuda'))  # batch size is 1
            entropy = Categorical(probs=torch.softmax(outputs.logits, -1)).entropy().tolist()[0]
            if entropy >= max_combin[0]:
                max_combin = [entropy, options]
    return max_combin[1][:-1]


In [ ]:
selection(context, question, answer, all_options)